Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.linear_algebra</a>
========

In [ ]:
import proveit
from proveit import a, b, c, d, e, f, g, i, IndexedVar
from proveit.logic import Equals
from proveit.numbers import one, two, three, four, five, Add, Interval, Mult, subtract, Sum
from proveit.linear_algebra import TensorProd, TensorExp
from proveit.linear_algebra.matrix_ops import ScalarProd
%begin demonstrations

## Miscellaneous Testing

The material below was developed to test various unification-related methods. Some of this material could be integrated into the `_demonstrations_` page eventually and/or deleted as development continues.

The `linear_algebra` package includes the following classes, some of which are only barely developed and might have no specific methods to be tested here (the MatrixProd and ScalarProd, for example, may eventually be handled as special cases of Mult):

• `SU` (stands for “special unitary”) — no methods<br/>
• `MatrixProd` — no methods except formatted()<br/>
• `ScalarProd` — no currently active methods<br/>
• `Matrix` and `ImproperMatrix` — no relevant methods except `Matrix.formatted()` and `ImproperMatrix.str()`<br/>
• `TensorProd` — methods: `TensorProd.factor()` and `TensorProd.distribute()`<br/>
• `TensorExp` — method: `TensorExp.do_reduced_simplification()`<br/>

### Testing the `TensorProd` class methods

The `TensorProd` class has the following class methods:<br/>
    <div style="width: 75%; border: 1px solid green; padding: 5px; margin: 20px; background-color: gainsboro; font-family:courier">factorization(self, **defaults_config)<br/>
    distribution(self, **defaults_config)</div>
</font>

### Some Example `TensorProd` For Testing

In [ ]:
tensor_prod_00 = TensorProd(ScalarProd(c, a), b)

In [ ]:
tensor_prod_000 = TensorProd(a, ScalarProd(c, b))

In [ ]:
tensor_prod_01 = TensorProd(ScalarProd(c, a), b, d)

In [ ]:
tensor_prod_02 = TensorProd(a, ScalarProd(c, b), d)

In [ ]:
tensor_prod_03 = TensorProd(a, b, ScalarProd(c, d))

In [ ]:
tensor_prod_04 = TensorProd(a, b, ScalarProd(c, d), e, f)

In [ ]:
tensor_prod_05 = TensorProd(a, b, ScalarProd(c, d), Add(one, e, one), f)

In [ ]:
tensor_prod_06 = TensorProd(Add(a, b), f)

In [ ]:
tensor_prod_07 = TensorProd(a, Add(b, c))

In [ ]:
tensor_prod_08 = TensorProd(a, b, Add(c, d, e), f)

In [ ]:
tensor_prod_with_sum_01 = TensorProd(a, Sum(i, IndexedVar(b, i), domain=Interval(one, three)), c)

In [ ]:
tensor_prod_with_sum_02 = TensorProd(Sum(i, IndexedVar(b, i), domain=Interval(two, four)), a)

In [ ]:
tensor_prod_with_sum_03 = TensorProd(a, Sum(i, IndexedVar(b, i), domain=Interval(two, four)))

In [ ]:
tensor_prod_with_sum_04 = TensorProd(a, b, c, d, Sum(i, IndexedVar(g, i), domain=Interval(one, five)), e)

### Testing the `TensorProd.factorization()` method

Factor the given scalar from one of the tensor product factors and return the original tensor product equal to the factored version. For example,<br/>

`TensorProd(a, cb, d).factorization(c)`<br/>

returns:<br/>

`|- TensorProd(a, ScalarProd(c, b), d).factorization(c) = c TensorProd(a, b, d)` <br/>
               
Note that this works only for explicit ScalarProd components within the TensorProd structure. Plans are to generalize this to not require an explicit ScalarProd object (instead allowing the more general Mult object).

In [ ]:
tensor_prod_00.factorization(c)

In [ ]:
tensor_prod_000.factorization(c)

In [ ]:
tensor_prod_01.factorization(c)

In [ ]:
tensor_prod_02.factorization(c)

In [ ]:
tensor_prod_03.factorization(c)

In [ ]:
tensor_prod_04.factorization(c)

In [ ]:
tensor_prod_05.factorization(c)

In [ ]:
try:
    tensor_prod_05.factorization(d)
    assert False, "Expecting a ValueError; should not get this far!"
except ValueError as the_error:
    print("ValueError: {}".format(the_error))

In [ ]:
try:
    tensor_prod_06.factorization(c)
    assert False, "Expecting a ValueError; should not get this far!"
except ValueError as the_error:
    print("ValueError: {}".format(the_error))

### Testing the `TensorProd.distribution()` method

Given a TensorProd factor at the (0-based) index location `idx` that is a sum or summation, distribute over that TensorProd factor and return an equality to the original TensorProd. For example, we could take the TensorProd $\text{tens_prod} = a \otimes (b + c) \otimes d$ and call `tens_prod.distribution(1)` to obtain:

$\vdash a \otimes (b + c) \otimes d = (a \otimes b \otimes d) + (a \otimes c \otimes d)$

In [ ]:
tensor_prod_05

In [ ]:
tensor_prod_05.distribution(3)

In [ ]:
tensor_prod_06

In [ ]:
tensor_prod_06.distribution(0)

In [ ]:
tensor_prod_07

In [ ]:
tensor_prod_07.distribution(1)

In [ ]:
tensor_prod_08

In [ ]:
tensor_prod_08.distribution(2)

In [ ]:
tensor_prod_with_sum_01

In [ ]:
tensor_prod_with_sum_01.distribution(1)

In [ ]:
tensor_prod_with_sum_02

In [ ]:
tensor_prod_with_sum_02.distribution(0)

In [ ]:
tensor_prod_with_sum_03

In [ ]:
tensor_prod_with_sum_03.distribution(1)

In [ ]:
tensor_prod_with_sum_04

In [ ]:
tensor_prod_with_sum_04.distribution(4)

In [ ]:
# recall one of our TensorProd objects without a sum or summation:
tensor_prod_02

In [ ]:
# we should get a meaningful error message when trying to distribute across
# a factor that is not a sum or summation:
try:
    tensor_prod_02.distribution(1)
    assert False, "Expecting a ValueError; should not get this far!"
except ValueError as the_error:
    print("ValueError: {}".format(the_error))

### Testing the `TensorProd.equate_factors()` method

Operating on the 'self' TensorProd and taking as an argument an equality between the 'self' TensorProd and another TensorProd with the same number of factors all but one of which agree with the factors of 'self', deduce and return a logical equivalence between the TensorProd equality and the equality of the two non-matching factors.

For example, letting $\text{tp_01} = a \otimes b \otimes c$ and $\text{tp_02} = a \otimes d \otimes c$, then calling

`tp.equate_factors(Equals(tp_01, tp_02))`

deduces and returns

$\vdash ((a \otimes b \otimes c) = (a \otimes d \otimes c)) \Leftrightarrow  (b = d)$

In [ ]:
tp_01, tp_02 = (TensorProd(a, b, c), TensorProd(a, d, c))

In [ ]:
tp_equality_01_02 = Equals(tp_01, tp_02)

In [ ]:
tp_01.equate_factors(tp_equality_01_02)

In [ ]:
tp_03, tp_04 = (TensorProd(a, b, Add(c, d), e), TensorProd(a, b, Add(one, two), e))

In [ ]:
tp_equality_03_04 = Equals(tp_03, tp_04)

In [ ]:
tp_03.equate_factors(tp_equality_03_04)

In [ ]:
%end demonstrations